In [48]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import nltk
from nltk.tokenize import RegexpTokenizer
import collections

In [49]:
data = pd.read_csv('data/results.csv')

In [50]:
'''
Total documents
Total word occurrences
Vocabulary size
Words occurring > 1000 times
Words occurring once
'''

total_documents = data.text.count()
# texts = ''.join(data.text).split()
texts = ''.join(data.text).lower()

In [51]:
# print(collections.Counter([x.lower() for x in ['a', 'b', 'c', 'a', 'b', 'b', 'A', 'B']]))
# pre process text

# REMOVER STOPWORDS?
# remover palavras maior que 3?
# remover palavras com acento?
# removido números

regex = RegexpTokenizer(r'\b[^\d\W]+')
tokens = regex.tokenize(texts)

In [ ]:
# considerar números??
# list(filter(lambda x:len(x)>3, tokens))

In [52]:
word_counter = collections.Counter(tokens)
total_word_occurrences = sum(word_counter.values())
vocabulary_size = len(word_counter.keys())

In [ ]:
most_frequent_words = len({x: word_counter[x] for x in word_counter if word_counter[x] > 1000})
less_frequent_words = len({x: word_counter[x] for x in word_counter if word_counter[x] == 1})

In [ ]:
table1 = pd.DataFrame(data={'names':[
                            'Total Documents', 
                            'Total word ocurrences', 
                            'Vocabulary size', 
                            'Words occurring > 1000 times', 
                            'Words occurring once'
                            ], 
                   'values':[
                           total_documents, 
                           total_word_occurrences, 
                           vocabulary_size, 
                           most_frequent_words, 
                           less_frequent_words
                           ]
                  })
print(table1)
# print(table1.to_string(index=False, header=False))

In [65]:
#QUESTAO 2
'''
word, freq, ranking, Pr(%), r.Pr
'''
ranking = 1
words, frequencies, rankings, probs, rprobs = [], [], [], [], []
for k,v in word_counter.most_common():
    prob = round((v/total_word_occurrences) * 100, 2)
    words.append(k)
    frequencies.append(v)
    rankings.append(ranking)
    probs.append(prob)
    rprobs.append(round(ranking * prob / 100, 3))
    ranking += 1

dataq2 = {'Word': words, 'Freq': frequencies, 'r': rankings, 'Pr(%)': probs, 'r.Pr': rprobs}
pd.DataFrame(dataq2).head(50)

,Word,Freq,r,Pr(%),r.Pr
0,de,10211,1,4.79,0.048
1,a,7499,2,3.52,0.070
2,o,6610,3,3.10,0.093
3,que,6554,4,3.08,0.123
4,e,5734,5,2.69,0.134
5,do,3465,6,1.63,0.098
6,em,3381,7,1.59,0.111
7,da,3012,8,1.41,0.113
8,um,2698,9,1.27,0.114
9,para,2323,10,1.09,0.109
